# Lado Servidor

## Importações

In [5]:
import socket
import sys
import random
from crc import CRC

## Funções Auxiliares

Nessa seção serão apresentadas algumas funções auxiliares que, em sua maioria, fazem operações simples mas que foram essências para o funcionamento correto do protocolo criado.

### Função "transformaEmBit"

Essa função transforma um array de bytes em uma sequência de bits. Cada byte da lista é codificado em oito bits, portanto zeros à esquerda podem ser adicionados para que o retorno fique sempre do tamanho correto.

In [6]:
# transforma um array de bytes em uma sequencia de bits
# entrada: lista de bytes
# retorno: sequência binária, onde cada byte da lista é codificada em oito bits
def transformaEmBit(listaBytes):
    resultado = "0b"

    for item in listaBytes:
        item = bin(int(str(item.hex()), 16))[2:]

        while(len(item) < 8):
            item = "0" + item
        resultado += item

    return resultado

Exemplo de funcionamento:

In [3]:
print(transformaEmBit([ GENTE COLOCA UM TESTE AQUI ]))

SyntaxError: invalid syntax (<ipython-input-3-6c48ec6e68aa>, line 1)

### Função "leBytes"

Essa função lê uma sequência de bytes transmitida por uma conexão. É necessário que seja enviado a quantidade de Bytes a serem lidos.

In [7]:
# recebe uma sequência de bytes de uma conexão socket
# entrada: conexão socket e a quantidade de bytes que serão recebidos
# retorno: lista de bytes
def leBytes(conexao, qtdBytes):
    listaBytes = []

    for i in range(qtdBytes):
        byteLido = conexao.recv(1)
        # adiciona os dados do cabeçalho a uma lista de bytes
        listaBytes.append(byteLido)

    return listaBytes

### Função "juntaBytes"

Essa função recebe uma lista de Bytes e os une, retornando uma sequência de Bytes concatenados.

In [8]:
# junta todos os bytes de uma lista de bytes
# entrada: uma lista de bytes
# retorno: uma sequência de bytes
def juntaBytes(listaDeBytes):
    resultado = b''

    for byte in listaDeBytes:
        resultado += byte

    return resultado

Exemplo de funcionamento:

In [ ]:
print(juntaBytes( [ GENTE COLOCAR UMA LISTA DE BYTES ] ))

## Servidor

No lado do servidor a primeira coisa feita é instanciar algumas constantes que são:
<ul>
    <li>**HOST** (endereço IP do servidor)</li>
    <li>**PORT** (a porta a qual ele escutará)</li>
</ul>
Depois disso, o socket do servidor é iniciado e ele entra em um loop para escutar a porta especificada. Quando um cliente envia uma requisição


In [ ]:
# import socket
def main(args):
    # flag delimitadora ("~")
    DELIMITADOR = bytes.fromhex('7e')

    # host padrão
    HOST = '127.0.0.1'

    # se os IPs foram passados como argumentos, são atribuídos às suas respectivas variáveis
#     if(len(args) > 1):
#         print(args[1])
#         HOST = args[1]
        
    PORT = 50017

    # inicializa o socket do servidor
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    sock.bind((HOST, PORT))
    sock.listen(1)

    while True:
        conexao, addr = sock.accept()

        fimMensagem = False

        # mensagem enviada pelo usuário
        mensagemCompleta = ''

        # o último número de sequência recebido
        ultimoRecebido = '0b10000000'

        print('')
        # o loop termina quando a mensagem for totalmente enviada
        while (not fimMensagem):
            cabecalho = b''
            cabecalho += conexao.recv(1)

            if(len(cabecalho) <= 0):
                fimMensagem = True
                continue

            # lista com todos os bytes recebidos
            listaBytes = leBytes(conexao, 10)

            # cabeçalho é o conjunto dos 11 primeiros bytes recebidos
            cabecalho += juntaBytes(listaBytes)

            # o tamanho da mensagem contida no quadro é o segundo byte
            tamanhoDados = cabecalho[1]

            # recebe os bytes da mensagem
            listaDados = leBytes(conexao, tamanhoDados)
            listaBytes += listaDados
            dados = b'' + juntaBytes(listaDados)

            # recebe o codigo CRC gerado pelo cliente
            # e o adiciona à lista de bytes
            codigoCRC = conexao.recv(1)
            listaBytes.append(codigoCRC)
            codigoCRC = conexao.recv(1)
            listaBytes.append(codigoCRC)

            # transforma a lista de bytes em sequências binárias, 
            # completando com zeros à esquerda para que sejam sequências de 8 bits
            mensagemBin = transformaEmBit(listaBytes)

            # simulador de erro de transmissão com 5% de chance de alteração de um bit do quadro
            if(random.random() > 0.95):
                print("Invertendo")
                posicao = random.randrange(0, tamanhoDados)
                mensagemBin = list(mensagemBin)
                mensagemBin[posicao] = bin(int(mensagemBin[posicao]) ^ 0x01)[2:]
            
            # verifica o CRC da mensagem
            crc = CRC()
            sequenciaACK = bytes([(cabecalho[2] & 0x80) + 1])

            # verifica se o número de sequência do último quadro recebido 
            # é diferente do quadro atual
            # também verifica (utilizando CRC) se os dados da mensagem foram corrompidos
            if((int(ultimoRecebido, 2) ^ (cabecalho[2] & 0x80)) and crc.verificaCRC(mensagemBin)):
                print("mensagem completa:", mensagemCompleta)
                mensagemCompleta += dados.decode("ascii")
                ultimoRecebido = bin(int(ultimoRecebido, 2) ^ 0x80)
            # caso alguma das duas verificações seja falsa, reenvia o ack do último quadro recebido
            else:
                sequenciaACK = bytes([int.from_bytes(sequenciaACK, byteorder='big') ^ 0x80])

            origem = bytes(cabecalho[3:7])
            destino = bytes(cabecalho[7:11])
            
            # monta o cabeçalho de confirmação
            confirmacao = DELIMITADOR + sequenciaACK + destino + origem

            #tenta enviar a confirmação, se ocorrer algum erro, ignora
            try:
                conexao.send(confirmacao)
            except:
                continue

        # escreve a mensagem recebida
        print("Mensagem:", mensagemCompleta)

        # tenta finalizar a conexão, caso ocorra um erro, ignora
        try:
            conexao.shutdown(socket.SHUT_WR)
        except:
            continue

        # fecha a conexão
        conexao.close()

main(sys.argv)


mensagem completa: 
Mensagem: oioioio
